In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
from opening_hours import OpeningHours
import re
module_path = os.path.abspath(os.path.join('../..', 'utils'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)


d:\CS\summer_project\comp47360-group8\ml_pipeline\utils


In [2]:
df_am = pd.read_csv('osm_places_amenity_cleaned.csv')
df_am.rename({'amenity':'subtype'}, axis=1, inplace=True)
df_sh = pd.read_csv('osm_places_leisure_cleaned.csv')
df_sh.rename({'shop':'subtype'}, axis=1, inplace=True)
df_le = pd.read_csv('osm_places_shop_cleaned.csv')
df_le.rename({'leisure':'subtype'}, axis=1, inplace=True)
df = pd.concat([df_am, df_sh, df_le])

In [3]:
del df_am
del df_sh
del df_le

In [4]:
df.head()

,id,lat,lon,addr:city,addr:housenumber,addr:postcode,addr:state,addr:street,subtype,cuisine,...,drink:wine,wheelchair,email,diet:vegan,diet:vegetarian,drink:coffee,drink:tea,contact:instagram,leisure,shop
0,349323821,40.762515,-73.976690,New York,43,10019.0,NY,West 55th Street,fast_food,burger,...,unknown,unknown,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN
1,357618253,40.740822,-73.784295,Fresh Meadows,61-09,11365.0,NY,190th Street,restaurant,burger,...,unknown,unknown,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN
2,357620442,40.741558,-73.978463,NaN,NaN,NaN,NaN,NaN,restaurant,korean,...,yes,unknown,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN
3,357621192,40.846563,-73.938127,New York,4140,10033.0,NY,Broadway,theatre,NaN,...,unknown,unknown,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN
4,368042980,40.721480,-73.997103,NaN,97,10012.0,NaN,Kenmare Street,arts_centre,NaN,...,unknown,unknown,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN


In [5]:
original_hours = df['opening_hours'].astype(str).values

In [6]:
# Split by ;
hours = [[hour.strip() for hour in hrs.split(';')] for hrs in original_hours ]

In [7]:
# Change shortenings of some days for the OpeningHours parser to work.
day_map = {
    "Mo":"Mon",
    "Tu":"Tue",
    "We":"Wed",
    "Fr": "Fri"
}


for i, hrs in enumerate(hours):
    for j, _ in enumerate(hrs):
        for key, val in day_map.items():
            hours[i][j] = hours[i][j].replace(key, val)

In [8]:
# Split by commas
hours = [[[hr_1.strip()  for hr_1 in hr.split(',')] for hr in hrs]for hrs in hours]

In [9]:

# Remove public holidays and summer holidays
# All have PH/SH in them
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        hours[i][j] = [hr_1 for hr_1 in hr if not re.search("PH|SH", hr_1)]

In [10]:
# Functions for determining if statements contain days
day_re = r"(Mon|Tue|Wed|Th|Fri|Sa|Su)(-(Mon|Tue|Wed|Th|Fri|Sa|Su))?"
single_day_re = r"(Mon|Tue|Wed|Th|Fri|Sa|Su)"
time_re = r"([01]\d|2[0-9]):([0-5]\d)(-([01]\d|2[0-9]):([0-5]\d))?|off|closed"
time_range_re = r"([01]\d|2[0-9]):([0-5]\d)(-([01]\d|2[0-9]):([0-5]\d))|off|closed"
def contains_days(string):
    if re.search(day_re, string):
        return True
    return False

def contains_times(string):
    if re.search(time_re, string):
        return True
    return False



In [11]:
for i,hrs in enumerate(hours):
    # If a time after a comma is missing the days, then we must give it the days from the comma before 
    # e.g. Mon-Th 11:00-12:00, 13:00-15:00 =>  Mon-Th 11:00-12:00, Mon-Th 13:00-15:00
    for j, hr in enumerate(hrs):
        prev_days = None
        for k, hr_1 in enumerate(hr):
            if contains_days(hr_1):
                prev_days = re.search(day_re, hr_1).group()
            elif prev_days:
                print("HERE", hours[i][j][k], prev_days)
                hours[i][j][k] = prev_days + " " + hours[i][j][k]
                print("HERE", hours[i][j][k], prev_days)

    
        # # If a day before a comma is missing the hours, then we must give it the days from the comma after 
        # # e.g. Mon-Th,Fri 13:00-15:00 =>  Mon-Th 13:00-15:00, Fri 13:00-15:00
        prev_hours = None
        for k, hr_1 in enumerate(hr[::-1]):
            if contains_times(hr_1):
                prev_hours = re.search(time_re, hr_1).group()
            elif prev_hours:
                print("HERE", hr_1, prev_hours)
                hours[i][j][-k-1] = hours[i][j][-k-1] + " " + prev_hours
                print("HERE", hr_1, prev_hours)


HERE 15:00-18:00 Mon-Fri
HERE Mon-Fri 15:00-18:00 Mon-Fri
HERE week 1-20 08:30-21:00
HERE week 1-20 08:30-21:00
HERE 17:30-01:00 Sa
HERE Sa 17:30-01:00 Sa
HERE 17:00-23:30 Su
HERE Su 17:00-23:30 Su
HERE 17:00-22:00 Mon-Th
HERE Mon-Th 17:00-22:00 Mon-Th
HERE 17:00-22:30 Fri-Sa
HERE Fri-Sa 17:00-22:30 Fri-Sa
HERE 17:00-22:00 Su
HERE Su 17:00-22:00 Su
HERE 17:00-22:00 Mon-Th
HERE Mon-Th 17:00-22:00 Mon-Th
HERE Sa 12:00-23:00
HERE Sa 12:00-23:00
HERE Sa 14:00-23:00
HERE Sa 14:00-23:00
HERE Fri 12:00-24:00
HERE Fri 12:00-24:00
HERE Sa 11:00-24:00
HERE Sa 11:00-24:00
HERE Fri 11:00-23:30
HERE Fri 11:00-23:30
HERE Mon 06:00-22:00
HERE Mon 06:00-22:00
HERE Mon-Th 08:00-03:00
HERE Mon-Th 08:00-03:00
HERE Fri 08:00-04:00
HERE Fri 08:00-04:00
HERE Fri 12:00-04:00
HERE Fri 12:00-04:00
HERE Mon-Th 08:00-23:00
HERE Mon-Th 08:00-23:00
HERE Fri 08:00-24:00
HERE Fri 08:00-24:00
HERE Sa 10:00-24:00
HERE Sa 10:00-24:00
HERE 17:00-22:00 Mon-Su
HERE Mon-Su 17:00-22:00 Mon-Su
HERE Sa 16:00-23:00
HERE Sa 16:

In [12]:
short_days = ["Mon", "Tue", "Wed", "Th", "Fri", "Sa", "Su"]

def increment_day(day: str):
    if day not in short_days:
        raise Exception(f"Error: no day in {day}")
    return short_days[short_days.index(day)+ 1]

def increment_days(days: str):
    """Increments a set of days, e.g. Mon => Tue, Mon-Fri => Tue-Sa"""
    new_days = re.match(day_re, days)
    if not new_days:
        return ""
    
    # Placeholder for sunday so we don't overlap
    placeholder = "TEMP_PLACEHOLDER"
    new_days = new_days.group().replace("Su", placeholder)

    # Replace Sa with Su, Fri with Sa and so on
    for i, day in enumerate(short_days[::-1]):
        new_days= new_days.replace(day, short_days[(7-i)%7])

    # Replace Sa (placeheld) with Mon
    return new_days.replace(placeholder, "Mon")
    


In [13]:
# Now convert the closing time to a 24 hour clock if >23:59
# e.g. 11:00-25:00 => 11:00-23:59, 00:00-01:00; Mon 11:00-28:00 => Mon 11:00-23:59, Tue 00:00-04:00
overtime_re = r"([01]\d|2[0-9]):([0-5]\d)(-(2[4-9]):([0-5]\d))"
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        for k, hr_1 in enumerate(hr):
            # Find overtime hours
            search = re.search(overtime_re, hr_1)
            if search:
                # Change current string to 
                before = hr_1[:search.start()+6]
                # Subtract 24 from the time that's >24
                # new_hour_string = days + 1, 00:00-hour_reduced
                hour_reduced = int(hr_1[search.start() + 6:search.start()+ 8]) - 24
                next_days = increment_days(hr_1)
                hours[i][j][k] = f"{before}23:59"
                hours[i][j] += [f"{next_days} 00:00-0{hour_reduced}:00"]
            

In [14]:
# Replace + symbols with -00:00 in the case where it's not already a time range 
# i.e. 11:00+ => 11:00-00:00
# But 11:00-23:00+ => 11:00-23:00
for i, hrs in enumerate(hours):
    for j, hr in enumerate(hrs):
        for k, hr_1 in enumerate(hr):
            if "+" not in hr_1:
                continue
            
            replace_str = "-00:00"
            if re.search(time_range_re, hr_1):
                replace_str = ""

            hours[i][j][k] = hours[i][j][k].replace("+", replace_str)

In [15]:
def get_all_days(days: str):
    """Converts days in a short-format day range into a list of days in long format.

    Args:
        days (str): Day range either as single day or range of days. e.g. "Mon", "Mon-Wed"

    Returns:
        List[str]: List of days in that day range in long format. e.g. ["monday"], ["monday", "tuesday", "wednesday"]
    """
    long_days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]

    # If "-" not in string then it's a single day
    if "-" not in days:
        return [long_days[short_days.index(days)]]
    

    output = []
    day_range = days.split("-")
    i = short_days.index(day_range[0])
    output += [long_days[i]]
    while short_days[i] != day_range[1]:
        i = (i + 1) % 7
        output += [long_days[i]]
    return output
    

def remove_days_off(day_str, opening_hours):
    """Removes days off from an opening string. Removes the off days in place.

    Args:
        day_str (str): string of days to remove in short format, e.g. "Mon", "Mon-We"
        opening_hours (list[dict[str]]): opening hours in OpeningHours format.
    """
    off_days = re.search(day_re, day_str)
    if not off_days:
        return
    
    off_days = off_days.group()
    days = get_all_days(off_days)
    
    opening_hours = [hour for hour in opening_hours if hour['day'] not in days]




In [16]:
print(hours[:10])

[[['Mon-Su 11:00-22:00']], [['nan']], [['Mon-Th 12:00-22:00'], ['Fri-Sa 12:00-23:00'], ['Su 12:00-22:00']], [['nan']], [['nan']], [['nan']], [['nan']], [['nan']], [['nan']], [['nan']]]


In [17]:
print(original_hours[:10])

['Mo-Su 11:00-22:00' 'nan'
 'Mo-Th 12:00-22:00; Fr-Sa 12:00-23:00; Su 12:00-22:00' 'nan' 'nan' 'nan'
 'nan' 'nan' 'nan' 'nan']


In [18]:
day_time_re = r"(Mon|Tue|Wed|Th|Fri|Sa|Su)?( *- *(Mon|Tue|Wed|Th|Fri|Sa|Su))? *([01]?\d|2[0-3]):([0-5]\d)( *- *([01]?\d|2[0-3]):([0-5]\d))"

no_re_count = 0 # No regex match count
exception_count = 0 # Exception count
opening_hours_parsed = []
for i, hrs in enumerate(hours):
    opening_hours = []
    days_off = []
    for hr in hrs:
        for hr_1 in hr:
            if hr_1.endswith("off"):
                days_off += [hr_1[:-4]]
                continue
            
            if hr_1.endswith("closed"):
                days_off += [hr_1[:-7]]
                continue

            parse_string = re.search(day_time_re, hr_1)
            
            if not parse_string:
                no_re_count += 1
                continue
            try:
                opening_hours += OpeningHours.parse(parse_string.group(), assume_type="24H").json()
            except:
                exception_count += 1
    
    for days in days_off:
        remove_days_off(days, opening_hours)
    
    opening_hours_parsed += [opening_hours]
    if i%10 == 0:
        print(i)


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [19]:
df['hours_parsed'] = opening_hours_parsed

In [20]:
df = df[['id', 'subtype', 'lat', 'lon', 'hours_parsed']]

In [21]:
df.to_csv("osm_opening_hours.csv", index=False) 